In [2]:
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

2024-03-05 10:57:40.411105: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 10:57:40.507009: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-05 10:57:40.903699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 10:57:40.903774: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 10:57:40.972158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [5]:
pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592284 sha256=fb50406a690f6c8b359f7c90c0b6781ce344d1262aafa97ddeb61a0621b0aad0
  Stored in directory: /home/sergi_carapuig/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pretty_midi as pm

file = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid"
extract_guitar_and_drums(file)

midi_stream = pm.PrettyMIDI(file)
midi_stream

In [34]:
import pretty_midi

def extract_guitar_and_drums(midi_file):
    
    midi_data = pretty_midi.PrettyMIDI(midi_file)

    for instrument in midi_data.instruments:
        if "guitar" == instrument.name.lower():   
            guitar_tracks = instrument
        elif "guitar 1" == instrument.name.lower():
            guitar_tracks = instrument
            
        if "drums" == instrument.name.lower():
            drum_tracks = instrument
        elif "%drum%" == instrument.name.lower():
            drum_tracks == instrument
    return guitar_tracks, drum_tracks

"""
# Example usage:
guitar_tracks, drum_tracks = extract_tracks_from_midi("example.mid")
print("Guitar Tracks:", guitar_tracks)
print("Drum Tracks:", drum_tracks)
"""


'\n# Example usage:\nguitar_tracks, drum_tracks = extract_tracks_from_midi("example.mid")\nprint("Guitar Tracks:", guitar_tracks)\nprint("Drum Tracks:", drum_tracks)\n'

In [35]:
type(extract_guitar_and_drums(file)[0])

pretty_midi.instrument.Instrument

In [27]:
midi_stream.

SyntaxError: invalid syntax (1056291145.py, line 1)

In [44]:
beat_times = midi_stream.get_beats()
beat_times

array([  0.      ,   0.582524,   1.165048,   1.747572,   2.330096,
         2.91262 ,   3.495144,   4.077668,   4.660192,   5.242716,
         5.82524 ,   6.407764,   6.990288,   7.572812,   8.155336,
         8.73786 ,   9.320384,   9.902908,  10.485432,  11.067956,
        11.65048 ,  12.233004,  12.815528,  13.398052,  13.980576,
        14.5631  ,  15.145624,  15.728148,  16.310672,  16.893196,
        17.47572 ,  18.058244,  18.640768,  19.223292,  19.805816,
        20.38834 ,  20.970864,  21.553388,  22.135912,  22.718436,
        23.30096 ,  23.883484,  24.466008,  25.048532,  25.631056,
        26.21358 ,  26.796104,  27.378628,  27.961152,  28.543676,
        29.1262  ,  29.708724,  30.291248,  30.873772,  31.456296,
        32.03882 ,  32.621344,  33.203868,  33.786392,  34.368916,
        34.95144 ,  35.533964,  36.116488,  36.699012,  37.281536,
        37.86406 ,  38.446584,  39.029108,  39.611632,  40.194156,
        40.77668 ,  41.359204,  41.941728,  42.524252,  43.106

In [45]:
import pretty_midi as pm

file = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid"

midi_data = pm.PrettyMIDI(file)

# Get the timestamps of each beat in the MIDI file
beat_times = midi_data.get_beats()

# Assuming 4/4 time signature, each bar contains four beats
# You can adjust this logic based on the time signature of your MIDI file
num_beats_per_bar = 4

# Initialize lists to store the start and end times of each bar
bars_start_times = []
bars_end_times = []

# Iterate through the beat times to extract bars
for i in range(0, len(beat_times), num_beats_per_bar):
    bar_start_time = beat_times[i]
    bar_end_time = beat_times[min(i + num_beats_per_bar, len(beat_times)) - 1]
    bars_start_times.append(bar_start_time)
    bars_end_times.append(bar_end_time)

# Print the start and end times of each bar
for bar_index, (start_time, end_time) in enumerate(zip(bars_start_times, bars_end_times)):
    print(f"Bar {bar_index + 1}: Start time = {start_time:.2f}, End time = {end_time:.2f}")


Bar 1: Start time = 0.00, End time = 1.75
Bar 2: Start time = 2.33, End time = 4.08
Bar 3: Start time = 4.66, End time = 6.41
Bar 4: Start time = 6.99, End time = 8.74
Bar 5: Start time = 9.32, End time = 11.07
Bar 6: Start time = 11.65, End time = 13.40
Bar 7: Start time = 13.98, End time = 15.73
Bar 8: Start time = 16.31, End time = 18.06
Bar 9: Start time = 18.64, End time = 20.39
Bar 10: Start time = 20.97, End time = 22.72
Bar 11: Start time = 23.30, End time = 25.05
Bar 12: Start time = 25.63, End time = 27.38
Bar 13: Start time = 27.96, End time = 29.71
Bar 14: Start time = 30.29, End time = 32.04
Bar 15: Start time = 32.62, End time = 34.37
Bar 16: Start time = 34.95, End time = 36.70
Bar 17: Start time = 37.28, End time = 39.03
Bar 18: Start time = 39.61, End time = 41.36
Bar 19: Start time = 41.94, End time = 43.69
Bar 20: Start time = 44.27, End time = 46.02
Bar 21: Start time = 46.60, End time = 48.35
Bar 22: Start time = 48.93, End time = 50.68
Bar 23: Start time = 51.26, 

In [47]:
import pretty_midi as pm

# Load MIDI file
file = "/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid"
midi_data = pm.PrettyMIDI(file)

# Get time signature changes
time_signature_changes = midi_data.time_signature_changes

# Define a function to get notes within each bar
def get_notes_in_bar(midi_data, bar_start_time, bar_end_time, instrument_name="guitar 1"):
    notes_in_bar = []
    for instrument in midi_data.instruments:
        if instrument.program == pm.instrument_name_to_program(instrument_name):
            for note in instrument.notes:
                if bar_start_time <= note.start < bar_end_time:
                    notes_in_bar.append(note)
    return notes_in_bar

# Iterate through time signature changes and extract guitar notes in each bar
for i in range(len(time_signature_changes)):
    if i == len(time_signature_changes) - 1:
        bar_end_time = midi_data.get_end_time()
    else:
        bar_end_time = time_signature_changes[i + 1].time
    bar_start_time = time_signature_changes[i].time
    bar_notes = get_notes_in_bar(midi_data, bar_start_time, bar_end_time)
    print(f"Bar {i+1}: {len(bar_notes)} guitar notes")

ValueError: guitar 1 is not a valid General MIDI instrument name.

In [8]:
midi_stream.get_end_time()

307.87000195833326

In [12]:
midi_stream.instruments

[Instrument(program=100, is_drum=False, name="Melody"),
 Instrument(program=91, is_drum=False, name="Background Vocals"),
 Instrument(program=33, is_drum=False, name="bass    "),
 Instrument(program=27, is_drum=False, name="guitar 1"),
 Instrument(program=27, is_drum=False, name="guitar 2"),
 Instrument(program=4, is_drum=False, name="e piano "),
 Instrument(program=12, is_drum=False, name="marimba "),
 Instrument(program=18, is_drum=False, name="organ   "),
 Instrument(program=88, is_drum=False, name="fantasy "),
 Instrument(program=0, is_drum=True, name="Snare Drum"),
 Instrument(program=0, is_drum=True, name="Drums")]

In [37]:
len(midi_stream.instruments[-2].notes)

244

In [20]:
notes_guitar = midi_stream.instruments[3].notes

In [29]:
notes_guitar

[Note(start=4.654124, end=6.077060, pitch=62, velocity=62),
 Note(start=4.669294, end=6.083128, pitch=67, velocity=92),
 Note(start=4.681430, end=6.083128, pitch=70, velocity=71),
 Note(start=4.617716, end=6.152910, pitch=50, velocity=58),
 Note(start=4.641988, end=6.183250, pitch=53, velocity=66),
 Note(start=4.587376, end=6.216623, pitch=43, velocity=58),
 Note(start=6.307643, end=6.583735, pitch=43, velocity=33),
 Note(start=6.395628, end=6.601939, pitch=53, velocity=74),
 Note(start=6.359220, end=6.611041, pitch=50, velocity=66),
 Note(start=6.429002, end=6.632278, pitch=69, velocity=74),
 Note(start=6.410798, end=6.638346, pitch=65, velocity=92),
 Note(start=6.401696, end=6.644414, pitch=60, velocity=74),
 Note(start=6.987254, end=8.841015, pitch=60, velocity=66),
 Note(start=7.008492, end=8.871355, pitch=63, velocity=83),
 Note(start=7.014560, end=8.871355, pitch=67, velocity=92),
 Note(start=6.941744, end=8.962374, pitch=43, velocity=50),
 Note(start=6.972084, end=9.038224, pitc

In [22]:
len(notes_guitar)

2851

In [25]:
notes_drum = midi_stream.instruments[10].notes

In [35]:
notes_drum

[Note(start=12.539436, end=12.606183, pitch=40, velocity=55),
 Note(start=12.609217, end=12.672931, pitch=40, velocity=44),
 Note(start=12.675965, end=12.685067, pitch=40, velocity=38),
 Note(start=12.736645, end=12.754848, pitch=40, velocity=51),
 Note(start=12.815528, end=12.821596, pitch=40, velocity=91),
 Note(start=12.970261, end=13.115892, pitch=40, velocity=109),
 Note(start=13.255455, end=13.401086, pitch=40, velocity=82),
 Note(start=13.404120, end=13.549751, pitch=40, velocity=109),
 Note(start=13.689314, end=13.834945, pitch=40, velocity=105),
 Note(start=13.868319, end=14.013950, pitch=40, velocity=73),
 Note(start=13.980576, end=14.016984, pitch=51, velocity=88),
 Note(start=13.980576, end=14.277906, pitch=35, velocity=100),
 Note(start=14.271838, end=14.338586, pitch=42, velocity=96),
 Note(start=14.469047, end=14.544896, pitch=42, velocity=73),
 Note(start=14.563100, end=14.708731, pitch=37, velocity=109),
 Note(start=14.857396, end=14.924144, pitch=42, velocity=109),
 N

In [34]:
notes_drum[0].pitch

40

In [26]:
len(notes_drum)

2829

In [27]:
notes_drum_2 = midi_stream.instruments[9].notes

In [28]:
len(notes_drum_2)

244

In [39]:
midi_stream.key_signature_changes

[KeySignature(key_number=10, time=0.0),
 KeySignature(key_number=11, time=179.41739199999998)]

In [40]:
midi_stream.get_tempo_changes()

(array([0.]), array([103.00004807]))

In [41]:
midi_stream.metadata

AttributeError: 'PrettyMIDI' object has no attribute 'metadata'

In [42]:
filename

NameError: name 'filename' is not defined

In [43]:
file

'/home/sergi_carapuig/code/MarkBerkovics/BandIt/raw_data/10cc_-_Dreadlock_Holiday.mid'

In [46]:
n = file.split("/")[-1].split(".")[0]
n

'10cc_-_Dreadlock_Holiday'

In [50]:
import os

song_name = os.path.splitext(os.path.basename(file))[0]
song_name


'10cc_-_Dreadlock_Holiday'

'10cc_-_Dreadlock_Holiday'

In [51]:
import pretty_midi

# Load MIDI file
midi_data = midi_stream

# Get time signature and tempo
time_signature = midi_data.time_signature_changes[0] if midi_data.time_signature_changes else None
tempo = midi_data.estimate_tempo()

if time_signature:
    # Calculate bar duration
    beats_per_bar = time_signature.numerator
    seconds_per_beat = 60.0 / tempo
    seconds_per_bar = beats_per_bar * seconds_per_beat

    # Segment the track into bars
    bars = []
    current_time = 0.0
    while current_time < midi_data.get_end_time():
        bars.append(midi_data.get_pitch(current_time, current_time + seconds_per_bar))
        current_time += seconds_per_bar

    # Output bars
    for i, bar in enumerate(bars):
        print(f"Bar {i+1}: {bar}")
else:
    print("No time signature found.")


AttributeError: 'PrettyMIDI' object has no attribute 'get_pitch'

In [53]:
midi_data.estimate_tempo()

229.0657439629829

In [56]:
midi_data.time_signature_changes[0]

TimeSignature(numerator=4, denominator=4, time=0.0)

In [52]:
import pretty_midi

# Load MIDI file
midi_data = midi_stream

# Get time signature and tempo
time_signature = midi_data.time_signature_changes[0] if midi_data.time_signature_changes else None
tempo = midi_data.estimate_tempo()

if time_signature:
    # Calculate bar duration
    beats_per_bar = time_signature.numerator
    seconds_per_beat = 60.0 / tempo
    seconds_per_bar = beats_per_bar * seconds_per_beat

    # Segment the track into bars
    bars = []
    current_time = 0.0
    while current_time < midi_data.get_end_time():
        bar_notes = midi_data.get_pitch_class_histogram(current_time, current_time + seconds_per_bar)
        bars.append(bar_notes)
        current_time += seconds_per_bar

    # Output bars
    for i, bar in enumerate(bars):
        print(f"Bar {i+1}: {bar}")
else:
    print("No time signature found.")



Bar 1: [0.07289896 0.08200121 0.09192887 0.1458539  0.0606868  0.06177307
 0.05063884 0.11449153 0.11408061 0.02708281 0.1038921  0.07467129]
Bar 2: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 3: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 4: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 5: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 6: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 7: [0.07686947 0.06578432 0.10914909 0.13545536 0.04063227 0.07468569
 0.05574413 0.1244229  0.09095485 0.02914085 0.12837339 0.06878769]
Bar 8: